# transitions_and_animation

> Transition and animation utilities for Tailwind CSS

In [ ]:
#| default_exp utilities.transitions_and_animation

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Optional, Dict, Any, Union, List
from cjm_fasthtml_tailwind.core.base import (
    SingleValueFactory, BaseFactory, BaseUtility, StandardUtility,
    combine_classes, TailwindValue, is_custom_property, is_arbitrary_value
)
from cjm_fasthtml_tailwind.builders.scales import (
    SimpleFactory, ScaledFactory, ScaleConfig, ScaledUtility
)

from fasthtml.common import Div
from fasthtml.jupyter import JupyUvi, HTMX
from cjm_fasthtml_tailwind.core.testing import create_test_app, create_test_page, start_test_server
from IPython.display import display

## Transition Property

Control which CSS properties transition:

In [ ]:
#| export
# Enhanced factory that supports both fixed values and custom properties
class TransitionPropertyFactory(SimpleFactory):
    """Factory for transition property utilities with custom value support."""
    
    def __call__(
        self,
        value: str  # Custom property or arbitrary value
    ) -> str:  # The formatted transition property CSS class
        """Handle custom transition properties."""
        if is_custom_property(value):
            return f"transition-({value})"
        elif is_arbitrary_value(value) or " " in value or "," in value:
            return f"transition-[{value}]"
        return f"transition-{value}"
    
    def get_info(
        self
    ) -> Dict[str, Any]:  # Dictionary with factory information
        """Get information about the transition property factory."""
        return {
            'description': self._doc,
            'valid_inputs': [
                'Access predefined transitions as attributes',
                'Call with custom property (e.g., "--my-transition")',
                'Call with arbitrary value (e.g., "width, height")'
            ],
            'options': {
                'predefined_values': list(self._values.keys()),
                'custom_usage': 'Call factory with property: transition("width, height")'
            }
        }

# Create transition property factory
transition = TransitionPropertyFactory(
    {
        "": "transition",  # Default transition
        "all": "transition-all",
        "colors": "transition-colors",
        "opacity": "transition-opacity",
        "shadow": "transition-shadow",
        "transform": "transition-transform",
        "none": "transition-none"
    },
    "Transition property utilities for controlling which CSS properties transition"
) # The transition property factory

In [ ]:
#| export
# Override __getattr__ to handle empty string key for default transition
class TransitionPropertyFactory(SimpleFactory):
    """Factory for transition property utilities with custom value support."""
    
    def __getattr__(
        self,
        name: str  # The attribute name to look up
    ) -> str:  # The corresponding CSS value from the dictionary
        """Get CSS value by attribute name, with special handling for default transition."""
        if name == "_default" or name == "default":
            return self._values[""]
        return super().__getattr__(name)
    
    def __call__(
        self,
        value: str  # Custom property or arbitrary value
    ) -> str:  # The formatted transition property CSS class
        """Handle custom transition properties."""
        if is_custom_property(value):
            return f"transition-({value})"
        # Always treat non-custom-property values as arbitrary
        return f"transition-[{value}]"
    
    def get_info(
        self
    ) -> Dict[str, Any]:  # Dictionary with factory information
        """Get information about the transition property factory."""
        return {
            'description': self._doc,
            'valid_inputs': [
                'Access predefined transitions as attributes',
                'Call with custom property (e.g., "--my-transition")',
                'Call with arbitrary value (e.g., "width, height")'
            ],
            'options': {
                'predefined_values': list(self._values.keys()),
                'custom_usage': 'Call factory with property: transition("width, height")'
            }
        }

# Recreate transition property factory with the enhanced class
transition = TransitionPropertyFactory(
    {
        "": "transition",  # Default transition
        "all": "transition-all",
        "colors": "transition-colors",
        "opacity": "transition-opacity",
        "shadow": "transition-shadow",
        "transform": "transition-transform",
        "none": "transition-none"
    },
    "Transition property utilities for controlling which CSS properties transition"
) # The transition property factory

In [ ]:
#|exports
def test_transitions_and_animation_property_examples():
    """Test transition property utilities."""
    # Test predefined transition properties
    assert str(transition.default) == "transition"
    assert str(transition.all) == "transition-all"
    assert str(transition.colors) == "transition-colors"
    assert str(transition.opacity) == "transition-opacity"
    assert str(transition.shadow) == "transition-shadow"
    assert str(transition.transform) == "transition-transform"
    assert str(transition.none) == "transition-none"
    
    # Test custom properties
    assert transition("--my-transition") == "transition-(--my-transition)"
    assert transition("--custom-props") == "transition-(--custom-props)"
    
    # Test arbitrary values
    assert transition("width") == "transition-[width]"
    assert transition("width, height") == "transition-[width, height]"
    assert transition("opacity 0.3s, transform 0.3s") == "transition-[opacity 0.3s, transform 0.3s]"

# Run the tests
test_transitions_and_animation_property_examples()

## Transition Behavior

Control the behavior of CSS transitions:

In [ ]:
#| export
transition_behavior = SimpleFactory(
    {
        "normal": "transition-normal",
        "discrete": "transition-discrete"
    },
    "Transition behavior utilities for controlling the behavior of CSS transitions"
) # The transition behavior factory

In [ ]:
#|exports
def test_transitions_and_animation_behavior_examples():
    """Test transition behavior utilities."""
    assert str(transition_behavior.normal) == "transition-normal"
    assert str(transition_behavior.discrete) == "transition-discrete"

# Run the tests
test_transitions_and_animation_behavior_examples()

## Transition Duration

Control the duration of CSS transitions:

In [ ]:
#| export
# Standard duration values in milliseconds
DURATION_SCALE = [0, 75, 100, 150, 200, 300, 500, 700, 1000] # Standard Tailwind duration scale

DURATION_CONFIG = ScaleConfig( # Duration configuration
    numeric=False,  # Don't use full numeric scale
    decimals=False,
    fractions=False,
    named=None,
    special={
        "0": "0",
        "75": "75",
        "100": "100",
        "150": "150",
        "200": "200",
        "300": "300",
        "500": "500",
        "700": "700",
        "1000": "1000",
        "initial": "initial"
    },
    negative=False
)

# Enhanced duration factory with custom value support
class DurationFactory(ScaledFactory):
    """Factory for duration utilities with millisecond support."""
    
    def __init__(
        self,
        prefix: str,
        config: ScaleConfig,
        doc: Optional[str] = None
    ):
        """Initialize duration factory with autocomplete properties."""
        super().__init__(prefix, config, doc)
        
        # Add duration properties for autocomplete
        self._0 = self(0)
        self._75 = self(75)
        self._100 = self(100)
        self._150 = self(150)
        self._200 = self(200)
        self._300 = self(300)
        self._500 = self(500)
        self._700 = self(700)
        self._1000 = self(1000)
        self.initial = self("initial")
    
    def __call__(
        self,
        value: Union[int, str],  # Duration in milliseconds or custom value
        negative: bool = False  # Not used for durations
    ) -> ScaledUtility:  # A duration utility instance
        """Create duration utility with value."""
        # Convert numeric values to strings
        str_value = str(value)
        
        # Check if it's a standard duration value
        if str_value in self.config.special:
            instance = ScaledUtility(self.prefix, self.config)
            instance._value = str_value
            return instance
        
        # Otherwise, create with standard formatting
        instance = ScaledUtility(self.prefix, self.config)
        instance._value = instance._format_value(value)
        return instance
    
    def get_info(
        self
    ) -> Dict[str, Any]:  # Dictionary with factory information
        """Get information about the duration factory."""
        return {
            'description': self._doc,
            'valid_inputs': [
                f'Standard durations: {", ".join(str(d) for d in DURATION_SCALE)}ms',
                'Access as attributes: duration._0, duration._75, etc.',
                'Call with milliseconds: duration(250)',
                'CSS custom properties: duration("--my-duration")',
                'Arbitrary values: duration("2s"), duration("100ms")'
            ],
            'options': {
                'prefix': self.prefix,
                'standard_scale': DURATION_SCALE
            }
        }

# Create duration factory
duration = DurationFactory("duration", DURATION_CONFIG, "Transition duration utilities for controlling the duration of CSS transitions") # The duration factory

In [ ]:
#|exports
def test_transitions_and_animation_duration_examples():
    """Test transition duration utilities."""
    # Test standard duration values with attributes
    assert str(duration._0) == "duration-0"
    assert str(duration._75) == "duration-75"
    assert str(duration._100) == "duration-100"
    assert str(duration._150) == "duration-150"
    assert str(duration._200) == "duration-200"
    assert str(duration._300) == "duration-300"
    assert str(duration._500) == "duration-500"
    assert str(duration._700) == "duration-700"
    assert str(duration._1000) == "duration-1000"
    assert str(duration.initial) == "duration-initial"
    
    # Test all attributes are available for autocomplete
    assert hasattr(duration, '_0')
    assert hasattr(duration, '_75')
    assert hasattr(duration, '_100')
    assert hasattr(duration, '_150')
    assert hasattr(duration, '_200')
    assert hasattr(duration, '_300')
    assert hasattr(duration, '_500')
    assert hasattr(duration, '_700')
    assert hasattr(duration, '_1000')
    assert hasattr(duration, 'initial')
    
    # Verify they are ScaledUtility instances
    assert isinstance(duration._300, ScaledUtility)
    assert isinstance(duration._500, ScaledUtility)
    
    # Test with numeric values
    assert str(duration(0)) == "duration-0"
    assert str(duration(75)) == "duration-75"
    assert str(duration(150)) == "duration-150"
    assert str(duration(1000)) == "duration-1000"
    
    # Test arbitrary values
    assert str(duration("250")) == "duration-[250]"
    assert str(duration("2s")) == "duration-[2s]"
    assert str(duration("100ms")) == "duration-[100ms]"
    assert str(duration("0.5s")) == "duration-[0.5s]"
    
    # Test custom properties
    assert str(duration("--my-duration")) == "duration-(--my-duration)"
    assert str(duration("--animation-speed")) == "duration-(--animation-speed)"

# Run the tests
test_transitions_and_animation_duration_examples()

## Transition Timing Function

Control the easing of CSS transitions:

In [ ]:
#| export
# Enhanced factory for easing functions with custom value support
class EaseFactory(SimpleFactory):
    """Factory for easing/timing function utilities with custom value support."""
    
    def __call__(
        self,
        value: str  # Custom property or arbitrary value
    ) -> str:  # The formatted ease CSS class
        """Handle custom easing functions."""
        if is_custom_property(value):
            return f"ease-({value})"
        elif is_arbitrary_value(value) or "(" in value or "," in value:
            return f"ease-[{value}]"
        return f"ease-{value}"
    
    def get_info(
        self
    ) -> Dict[str, Any]:  # Dictionary with factory information
        """Get information about the ease factory."""
        return {
            'description': self._doc,
            'valid_inputs': [
                'Access predefined easings as attributes',
                'Call with custom property (e.g., "--my-ease")',
                'Call with cubic-bezier: ease("cubic-bezier(0.4, 0, 0.2, 1)")'
            ],
            'options': {
                'predefined_values': list(self._values.keys()),
                'custom_usage': 'Call factory with easing: ease("cubic-bezier(0.4, 0, 0.2, 1)")'
            }
        }

# Create ease factory
ease = EaseFactory(
    {
        "linear": "ease-linear",
        "_in": "ease-in",
        "out": "ease-out",
        "in-out": "ease-in-out",
        "initial": "ease-initial"
    },
    "Transition timing function utilities for controlling the easing of CSS transitions"
) # The ease factory

In [ ]:
#|exports
def test_transitions_and_animation_timing_examples():
    """Test transition timing function utilities."""
    # Test predefined easing functions
    assert str(ease.linear) == "ease-linear"
    assert str(ease._in) == "ease-in"  # Note: 'in' is a Python keyword, so we use 'in_'
    assert str(ease.out) == "ease-out"
    assert str(ease.in_out) == "ease-in-out"
    assert str(ease.initial) == "ease-initial"
    
    # Test custom properties
    assert ease("--my-ease") == "ease-(--my-ease)"
    assert ease("--custom-timing") == "ease-(--custom-timing)"
    
    # Test cubic-bezier functions
    assert ease("cubic-bezier(0.4, 0, 0.2, 1)") == "ease-[cubic-bezier(0.4, 0, 0.2, 1)]"
    assert ease("cubic-bezier(0.1, 0.7, 1.0, 0.1)") == "ease-[cubic-bezier(0.1, 0.7, 1.0, 0.1)]"
    
    # Test steps function
    assert ease("steps(4, jump-start)") == "ease-[steps(4, jump-start)]"

# Run the tests
test_transitions_and_animation_timing_examples()

## Transition Delay

Control the delay of CSS transitions:

In [ ]:
#| export
# Standard delay values in milliseconds (same as duration scale)
DELAY_SCALE = [0, 75, 100, 150, 200, 300, 500, 700, 1000] # Standard Tailwind delay scale

DELAY_CONFIG = ScaleConfig( # Delay configuration
    numeric=False,  # Don't use full numeric scale
    decimals=False,
    fractions=False,
    named=None,
    special={
        "0": "0",
        "75": "75",
        "100": "100",
        "150": "150",
        "200": "200",
        "300": "300",
        "500": "500",
        "700": "700",
        "1000": "1000"
    },
    negative=False
)

# Enhanced delay factory with custom value support
class DelayFactory(DurationFactory):
    """Factory for delay utilities with millisecond support."""
    
    def __init__(
        self,
        prefix: str,
        config: ScaleConfig,
        doc: Optional[str] = None
    ):
        """Initialize delay factory with autocomplete properties."""
        # Call parent init but skip the 'initial' property
        ScaledFactory.__init__(self, prefix, config, doc)
        
        # Add delay properties for autocomplete
        self._0 = self(0)
        self._75 = self(75)
        self._100 = self(100)
        self._150 = self(150)
        self._200 = self(200)
        self._300 = self(300)
        self._500 = self(500)
        self._700 = self(700)
        self._1000 = self(1000)
        # Note: delay doesn't have 'initial' like duration does

# Create delay factory
delay = DelayFactory("delay", DELAY_CONFIG, "Transition delay utilities for controlling the delay of CSS transitions") # The delay factory

In [ ]:
#|exports
def test_transitions_and_animation_delay_examples():
    """Test transition delay utilities."""
    # Test standard delay values with attributes
    assert str(delay._0) == "delay-0"
    assert str(delay._75) == "delay-75"
    assert str(delay._100) == "delay-100"
    assert str(delay._150) == "delay-150"
    assert str(delay._200) == "delay-200"
    assert str(delay._300) == "delay-300"
    assert str(delay._500) == "delay-500"
    assert str(delay._700) == "delay-700"
    assert str(delay._1000) == "delay-1000"
    
    # Test all attributes are available for autocomplete
    assert hasattr(delay, '_0')
    assert hasattr(delay, '_75')
    assert hasattr(delay, '_100')
    assert hasattr(delay, '_150')
    assert hasattr(delay, '_200')
    assert hasattr(delay, '_300')
    assert hasattr(delay, '_500')
    assert hasattr(delay, '_700')
    assert hasattr(delay, '_1000')
    
    # Verify they are ScaledUtility instances
    assert isinstance(delay._300, ScaledUtility)
    assert isinstance(delay._500, ScaledUtility)
    
    # Test with numeric values
    assert str(delay(0)) == "delay-0"
    assert str(delay(75)) == "delay-75"
    assert str(delay(150)) == "delay-150"
    assert str(delay(1000)) == "delay-1000"
    
    # Test arbitrary values
    assert str(delay("250")) == "delay-[250]"
    assert str(delay("2s")) == "delay-[2s]"
    assert str(delay("100ms")) == "delay-[100ms]"
    assert str(delay("0.5s")) == "delay-[0.5s]"
    
    # Test custom properties
    assert str(delay("--my-delay")) == "delay-(--my-delay)"
    assert str(delay("--animation-delay")) == "delay-(--animation-delay)"

# Run the tests
test_transitions_and_animation_delay_examples()

## Animation

Utilities for animating elements with CSS animations:

In [ ]:
#| export
# Enhanced factory for animations with custom value support
class AnimationFactory(SimpleFactory):
    """Factory for animation utilities with custom value support."""
    
    def __call__(
        self,
        value: str  # Custom property or arbitrary animation value
    ) -> str:  # The formatted animation CSS class
        """Handle custom animations."""
        if is_custom_property(value):
            return f"animate-({value})"
        elif is_arbitrary_value(value) or " " in value:
            return f"animate-[{value}]"
        return f"animate-{value}"
    
    def get_info(
        self
    ) -> Dict[str, Any]:  # Dictionary with factory information
        """Get information about the animation factory."""
        return {
            'description': self._doc,
            'valid_inputs': [
                'Access predefined animations as attributes',
                'Call with custom property (e.g., "--my-animation")',
                'Call with arbitrary value (e.g., "slide 1s ease-in-out")'
            ],
            'options': {
                'predefined_values': list(self._values.keys()),
                'custom_usage': 'Call factory with animation: animate("slide 1s ease-in-out")'
            }
        }

# Create animation factory
animate = AnimationFactory(
    {
        "spin": "animate-spin",
        "ping": "animate-ping",
        "pulse": "animate-pulse",
        "bounce": "animate-bounce",
        "none": "animate-none"
    },
    "Animation utilities for animating elements with CSS animations"
) # The animation factory

In [ ]:
#|exports
def test_transitions_and_animation_examples():
    """Test animation utilities."""
    # Test predefined animations
    assert str(animate.spin) == "animate-spin"
    assert str(animate.ping) == "animate-ping"
    assert str(animate.pulse) == "animate-pulse"
    assert str(animate.bounce) == "animate-bounce"
    assert str(animate.none) == "animate-none"
    
    # Test custom properties
    assert animate("--my-animation") == "animate-(--my-animation)"
    assert animate("--custom-spin") == "animate-(--custom-spin)"
    
    # Test arbitrary animation values
    assert animate("slide 1s ease-in-out") == "animate-[slide 1s ease-in-out]"
    assert animate("fade 0.5s linear infinite") == "animate-[fade 0.5s linear infinite]"
    assert animate("wiggle 1s ease-in-out infinite") == "animate-[wiggle 1s ease-in-out infinite]"

# Run the tests
test_transitions_and_animation_examples()

## Practical Examples

Let's see how to use these transition and animation utilities in real FastHTML components:

In [ ]:
#|exports
def test_transitions_and_animation_fasthtml_examples():
    """Test transition and animation utilities in practical FastHTML component examples."""
    from fasthtml.common import Div, Button, Card, Span, A, Img
    from cjm_fasthtml_tailwind.utilities.backgrounds import bg
    from cjm_fasthtml_tailwind.utilities.typography import font_size, text_color
    from cjm_fasthtml_tailwind.utilities.spacing import p
    from cjm_fasthtml_tailwind.utilities.borders import rounded
    from cjm_fasthtml_tailwind.utilities.effects import shadow
    from cjm_fasthtml_tailwind.utilities.transforms import scale_tw, translate
    from cjm_fasthtml_tailwind.utilities.effects import opacity
    from cjm_fasthtml_tailwind.utilities.transforms import transform
    from cjm_fasthtml_tailwind.utilities.layout import display_tw
    from cjm_fasthtml_tailwind.utilities.flexbox_and_grid import gap
    
    # Button with hover transition
    button = Button(
        "Hover me",
        cls=combine_classes(
            bg.blue._500, bg.blue._700.hover, text_color.white, p.x(4), p.y(2), rounded.full,
            transition.colors,
            duration._300,
            ease.in_out
        )
    )
    assert "transition-colors" in button.attrs['class']
    assert "duration-300" in button.attrs['class']
    assert "ease-in-out" in button.attrs['class']
    
    # Card with shadow transition
    card = Card(
        "Content",
        cls=combine_classes(
            shadow.md, shadow.xl.hover,
            transition.shadow,
            duration._200
        )
    )
    assert "transition-shadow" in card.attrs['class']
    assert "duration-200" in card.attrs['class']
    
    # Scale transform on hover
    scale_div = Div(
        "Scale on hover",
        cls=combine_classes(
            scale_tw(110).hover,
            transition.transform,
            duration._150,
            ease.out
        )
    )
    assert "transition-transform" in scale_div.attrs['class']
    assert "duration-150" in scale_div.attrs['class']
    assert "ease-out" in scale_div.attrs['class']
    
    # Delayed transition
    delayed_link = A(
        "Delayed hover",
        cls=combine_classes(
            text_color.gray._600, text_color.blue._500.hover,
            transition.colors,
            duration._500,
            delay._200
        )
    )
    assert "delay-200" in delayed_link.attrs['class']
    
    # Multiple property transitions
    multi_transition = Div(
        "Multiple transitions",
        cls=combine_classes(
            opacity._50, opacity._100.hover, transform.gpu, translate.y(1).hover,
            transition("opacity, transform"),
            duration._300
        )
    )
    assert "transition-[opacity, transform]" in multi_transition.attrs['class']
    
    # Return all examples in a grid layout
    return Div(
        button,
        card,
        scale_div,
        delayed_link,
        multi_transition,
        cls=combine_classes(display_tw.grid, gap(5))
    )

# Run the tests
test_transitions_and_animation_fasthtml_examples()

```html
<div class="grid gap-5">
<button class="bg-blue-500 hover:bg-blue-700 text-white px-4 py-2 rounded-full transition-colors duration-300 ease-in-out">Hover me</button>  <article class="shadow-md hover:shadow-xl transition-shadow duration-200">Content</article>
  <div class="hover:scale-110 transition-transform duration-150 ease-out">Scale on hover</div>
<a href="#" class="text-gray-600 hover:text-blue-500 transition-colors duration-500 delay-200">Delayed hover</a>  <div class="opacity-50 hover:opacity-100 transform-gpu hover:translate-y-1 transition-[opacity, transform] duration-300">Multiple transitions</div>
</div>

```

In [ ]:
#| eval: false
#| output: false
test_func = test_transitions_and_animation_fasthtml_examples
app, rt = create_test_app()

@rt
def index():
    return create_test_page(test_func.__doc__.title().replace('.', ''), test_func())
server = start_test_server(app)
display(HTMX())
server.stop()

In [ ]:
#|exports
def test_transitions_and_animation_animation_fasthtml_examples():
    """Test animation utilities in practical FastHTML component examples."""
    from fasthtml.common import Div, Span, Button
    from cjm_fasthtml_tailwind.utilities.borders import border, border_color, rounded
    from cjm_fasthtml_tailwind.utilities.sizing import w, h
    from cjm_fasthtml_tailwind.utilities.backgrounds import bg
    from cjm_fasthtml_tailwind.utilities.layout import position, top, right, display_tw
    from cjm_fasthtml_tailwind.utilities.typography import font_size, text_color
    from cjm_fasthtml_tailwind.utilities.flexbox_and_grid import gap
    
    # Loading spinner
    spinner = Div(
        cls=combine_classes(
            border._2, border_color.gray._200, border_color.t.blue._500, rounded.full, w(6), h(6),
            animate.spin
        )
    )
    assert "animate-spin" in spinner.attrs['class']
    
    # Ping notification
    notification_dot = Span(
        cls=combine_classes(
            position.absolute, top.negative(1), right.negative(1), h(3), w(3), bg.red._500, rounded.full,
            animate.ping
        )
    )
    assert "animate-ping" in notification_dot.attrs['class']
    
    # Pulse skeleton loader
    skeleton = Div(
        cls=combine_classes(
            h(4), bg.gray._200, rounded.full,
            animate.pulse
        )
    )
    assert "animate-pulse" in skeleton.attrs['class']
    
    # Bounce arrow
    arrow = Div(
        "↓",
        cls=combine_classes(
            font_size._2xl,
            animate.bounce
        )
    )
    assert "animate-bounce" in arrow.attrs['class']
    
    # Custom animation
    custom_animated = Div(
        "Custom animation",
        cls=combine_classes(
            animate("slide-in 0.5s ease-out forwards")
        )
    )
    assert "animate-[slide-in 0.5s ease-out forwards]" in custom_animated.attrs['class']
    
    # Animation with CSS variable
    var_animated = Div(
        "Variable animation",
        cls=combine_classes(
            animate("--custom-fade-in")
        )
    )
    assert "animate-(--custom-fade-in)" in var_animated.attrs['class']
    
    # Return all examples in a grid layout
    return Div(
        spinner,
        notification_dot,
        skeleton,
        arrow,
        custom_animated,
        var_animated,
        cls=combine_classes(display_tw.grid, gap(5))
    )

# Run the tests
test_transitions_and_animation_animation_fasthtml_examples()

```html
<div class="grid gap-5">
  <div class="border-2 border-gray-200 border-t-blue-500 rounded-full w-6 h-6 animate-spin"></div>
<span class="absolute -top-1 -right-1 h-3 w-3 bg-red-500 rounded-full animate-ping"></span>  <div class="h-4 bg-gray-200 rounded-full animate-pulse"></div>
  <div class="text-2xl animate-bounce">↓</div>
  <div class="animate-[slide-in 0.5s ease-out forwards]">Custom animation</div>
  <div class="animate-(--custom-fade-in)">Variable animation</div>
</div>

```

In [ ]:
#| eval: false
#| output: false
test_func = test_transitions_and_animation_animation_fasthtml_examples
app, rt = create_test_app()

@rt
def index():
    return create_test_page(test_func.__doc__.title().replace('.', ''), test_func())
server = start_test_server(app)
display(HTMX())
server.stop()

In [ ]:
#|exports
def test_transitions_and_animation_composition_fasthtml_examples():
    """Test composing multiple transition utilities together."""
    from fasthtml.common import Div, Button
    from cjm_fasthtml_tailwind.utilities.backgrounds import bg_linear, from_color, to_color
    from cjm_fasthtml_tailwind.utilities.transforms import scale_tw, translate, transform
    from cjm_fasthtml_tailwind.utilities.effects import shadow, opacity
    from cjm_fasthtml_tailwind.utilities.layout import display_tw
    from cjm_fasthtml_tailwind.utilities.flexbox_and_grid import gap
    
    # Complete transition setup
    interactive_element = Button(
        "Interactive",
        cls=combine_classes(
            bg_linear.to_r, from_color.blue._500, to_color.purple._600,
            from_color.blue._600.hover, to_color.purple._700.hover,
            transform.gpu, scale_tw(105).hover, scale_tw(95).active,
            shadow.lg, shadow.xl.hover,
            transition.all,  # Transition all properties
            duration._300,
            ease.in_out,
            delay._0
        )
    )
    assert "transition-all" in interactive_element.attrs['class']
    assert "duration-300" in interactive_element.attrs['class']
    assert "ease-in-out" in interactive_element.attrs['class']
    assert "delay-0" in interactive_element.attrs['class']
    
    # Different transitions for different properties
    multi_property = Div(
        "Multi-property transitions",
        cls=combine_classes(
            opacity._75, opacity._100.hover,
            transform.gpu, translate.x(2).hover,
            transition("opacity, transform"),
            duration("200ms"),  # Custom duration
            ease("cubic-bezier(0.4, 0, 0.2, 1)")  # Custom easing
        )
    )
    assert "transition-[opacity, transform]" in multi_property.attrs['class']
    assert "duration-[200ms]" in multi_property.attrs['class']
    assert "ease-[cubic-bezier(0.4, 0, 0.2, 1)]" in multi_property.attrs['class']
    
    # Combining animation with transitions
    animated_transitional = Div(
        "Loading...",
        cls=combine_classes(
            animate.pulse,
            opacity._50.hover,
            transition.opacity,
            duration._150
        )
    )
    assert "animate-pulse" in animated_transitional.attrs['class']
    assert "transition-opacity" in animated_transitional.attrs['class']
    
    # Return all examples in a grid layout
    return Div(
        interactive_element,
        multi_property,
        animated_transitional,
        cls=combine_classes(display_tw.grid, gap(5))
    )

# Run the tests
test_transitions_and_animation_composition_fasthtml_examples()

```html
<div class="grid gap-5">
<button class="bg-linear-to-r from-blue-500 to-purple-600 hover:from-blue-600 hover:to-purple-700 transform-gpu hover:scale-105 active:scale-95 shadow-lg hover:shadow-xl transition-all duration-300 ease-in-out delay-0">Interactive</button>  <div class="opacity-75 hover:opacity-100 transform-gpu hover:translate-x-2 transition-[opacity, transform] duration-[200ms] ease-[cubic-bezier(0.4, 0, 0.2, 1)]">Multi-property transitions</div>
  <div class="animate-pulse hover:opacity-50 transition-opacity duration-150">Loading...</div>
</div>

```

In [ ]:
#| eval: false
#| output: false
test_func = test_transitions_and_animation_composition_fasthtml_examples
app, rt = create_test_app()

@rt
def index():
    return create_test_page(test_func.__doc__.title().replace('.', ''), test_func())
server = start_test_server(app)
display(HTMX())
server.stop()

## Documentation Tests

Test that all factories provide proper documentation:

In [ ]:
#| export
def test_transitions_and_animation_factory_documentation():
    """Test that transition and animation factories have accessible documentation."""
    # Test transition property factory
    assert transition.describe() == "Transition property utilities for controlling which CSS properties transition"
    transition_info = transition.get_info()
    assert 'predefined_values' in transition_info['options']
    assert '' in transition_info['options']['predefined_values']  # Default transition
    assert 'all' in transition_info['options']['predefined_values']
    
    # Test transition behavior factory
    assert transition_behavior.describe() == "Transition behavior utilities for controlling the behavior of CSS transitions"
    behavior_info = transition_behavior.get_info()
    assert 'available_values' in behavior_info['options']
    assert 'normal' in behavior_info['options']['available_values']
    assert 'discrete' in behavior_info['options']['available_values']
    
    # Test duration factory
    assert duration.describe() == "Transition duration utilities for controlling the duration of CSS transitions"
    duration_info = duration.get_info()
    assert 'standard_scale' in duration_info['options']
    assert duration_info['options']['standard_scale'] == DURATION_SCALE
    
    # Test ease factory
    assert ease.describe() == "Transition timing function utilities for controlling the easing of CSS transitions"
    ease_info = ease.get_info()
    assert 'predefined_values' in ease_info['options']
    assert 'linear' in ease_info['options']['predefined_values']
    assert 'in-out' in ease_info['options']['predefined_values']
    
    # Test delay factory
    assert delay.describe() == "Transition delay utilities for controlling the delay of CSS transitions"
    delay_info = delay.get_info()
    assert 'standard_scale' in delay_info['options']
    
    # Test animation factory
    assert animate.describe() == "Animation utilities for animating elements with CSS animations"
    animate_info = animate.get_info()
    assert 'predefined_values' in animate_info['options']
    assert 'spin' in animate_info['options']['predefined_values']
    assert 'bounce' in animate_info['options']['predefined_values']

# Run the tests
test_transitions_and_animation_factory_documentation()

In [ ]:
#| eval: false
#| output: false
test_func = test_transitions_and_animation_composition_fasthtml_examples
app, rt = create_test_app()

@rt
def index():
    return create_test_page(test_func.__doc__.title().replace('.', ''), test_func())
server = start_test_server(app)
display(HTMX())
server.stop()

## Helper Functions

Convenient functions for common transition and animation patterns:

In [ ]:
#| export
def smooth_transition(
    properties: str = "all",  # Which properties to transition (default: all)
    duration_ms: int = 300,  # Duration in milliseconds
    easing: str = "in-out"  # Easing function
) -> str:  # Combined CSS classes for smooth transitions
    """Create a smooth transition with common defaults."""
    # Map property names to transition utilities
    property_map = {
        "all": transition.all,
        "colors": transition.colors,
        "opacity": transition.opacity,
        "shadow": transition.shadow,
        "transform": transition.transform
    }
    
    # Get transition property
    trans_prop = property_map.get(properties, transition(properties))
    
    # Map easing names
    easing_map = {
        "linear": ease.linear,
        "in": ease._in,
        "out": ease.out,
        "in-out": ease.in_out
    }
    
    # Get easing function
    ease_func = easing_map.get(easing, ease(easing))
    
    return combine_classes(
        trans_prop,
        duration(duration_ms),
        ease_func
    )

In [ ]:
#| export
def hover_effect(
    duration_ms: int = 200  # Duration in milliseconds
) -> str:  # Combined CSS classes for hover effects
    """Standard hover effect transition for interactive elements."""
    return combine_classes(
        transition.all,
        duration(duration_ms),
        ease.in_out
    )

In [ ]:
#| export
def fade_in(
    duration_ms: int = 500,  # Duration in milliseconds
    delay_ms: Optional[int] = None  # Optional delay in milliseconds
) -> str:  # Combined CSS classes for fade-in effect
    """Fade-in transition for entering elements."""
    classes = [
        transition.opacity,
        duration(duration_ms),
        ease.out
    ]
    
    if delay_ms is not None:
        classes.append(delay(delay_ms))
    
    return combine_classes(*classes)

In [ ]:
#| export
def loading_spinner(
) -> str:  # Combined CSS classes for a loading spinner
    """Create a loading spinner animation."""
    from cjm_fasthtml_tailwind.utilities.borders import border, border_color, rounded
    return combine_classes(
        border._2, border_color.gray._200, border_color.t.current, rounded.full,
        animate.spin
    )

In [ ]:
#| export
def skeleton_loader(
) -> str:  # Combined CSS classes for a skeleton loader
    """Create a skeleton loader animation for content placeholders."""
    from cjm_fasthtml_tailwind.utilities.backgrounds import bg
    return combine_classes(
        bg.gray._200,
        animate.pulse
    )

In [ ]:
#|exports
def test_transitions_and_animation_helper_examples():
    """Test helper functions for common transition patterns."""
    # Test smooth_transition
    assert smooth_transition() == "transition-all duration-300 ease-in-out"
    assert smooth_transition("colors", 200, "out") == "transition-colors duration-200 ease-out"
    assert smooth_transition("opacity, transform", 500, "linear") == "transition-[opacity, transform] duration-500 ease-linear"
    
    # Test hover_effect
    assert hover_effect() == "transition-all duration-200 ease-in-out"
    assert hover_effect(150) == "transition-all duration-150 ease-in-out"
    
    # Test fade_in
    assert fade_in() == "transition-opacity duration-500 ease-out"
    assert fade_in(300) == "transition-opacity duration-300 ease-out"
    assert fade_in(500, 200) == "transition-opacity duration-500 ease-out delay-200"
    
    # Test loading_spinner
    assert loading_spinner() == "border-2 border-gray-200 border-t-current rounded-full animate-spin"
    
    # Test skeleton_loader
    assert skeleton_loader() == "bg-gray-200 animate-pulse"

# Run the tests
test_transitions_and_animation_helper_examples()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()